# The method behind econpizza with heterogenous agents

The methodology extends Auclert et al. (2022, ECMA) to fully nonlinear functions. Let, for the sake of generality, letters with a tilde denote the complete expected time series of a variable up to some distant point in time $T$, e.g $\tilde{x} = \{x_{t}\}_{t=0}^T$.
Most heterogeneous agent models can be expressed in terms of the three functions $V$, $D$ and $F$:

$$
\begin{align}
v_t =& V(\tilde{x}, v_{t+1})\\
d_t =& D(\tilde{x}, \tilde{v}, d_{t-1})\\
0 =& F(\tilde{x}, \tilde{d}, \tilde{v})
\end{align}
$$

where $x_t$ is the time-$t$ vector of _aggregate_ variables, $v_t$ is the agents decision functions over the complete state space and $d_t$ is the distribution accross agents.

Let further the steady state values be denoted by a bar, i.e. $\bar{x}$, $\bar{d}$ and $\bar{v}$.

$V$ often take the form of a value function. Note that the above assumes that $V$ is independent of the distribution at any point in time. Assuming that after $T$ we are back in the steady state, the sequence $\tilde{v}$ can be found by starting with
$$
v_T = V(\tilde{x}, \bar{v})
$$
and iterating backwards until we arrive at $v_0$.

Now taking $v_0$ as given and assuming we are also in the steady state in $t=0$, we can start with $d_{-1}=\bar{d}$ and iterate $D$ forward:

$$
d_0 = D(\tilde{x}, \tilde{v}, \bar{d})
$$
until we arrive at $d_T$. This yields $\tilde{d}$.

Denote the function that generates $\tilde{v}$ from $\tilde{x}$ as $\hat{V}$, and a function that generates $\tilde{d}$ from $\tilde{x}$ and $\tilde{v}$ as $\hat{D}$. Then we can define
$$
G(\tilde{x}) = F\left(\tilde{x}, \hat{D}\left(\tilde{x}, \hat{V}(\tilde{x})\right), \hat{V}(\tilde{x})\right) = 0
$$
as a function that only depends on the sequence $\tilde{x}$.

Given that the jacobian $J_G$ of $G(\cdot)$ is available via automatic differentiation, we can then use a Newton method to solve for $\tilde{x}$ directly without having to explicitely keep track of any distribution variable.